# Data Prep

In [ ]:
import numpy as np
import pandas as pd
import os
import pydicom
pd.set_option('display.max_colwidth',None)

In [ ]:
img_train_path = './data/train/'
img_test_path = './data/test/'
train_csv_path = './data/train.csv'
new_csv_path = './data/new_train.csv'

In [ ]:
train_df = pd.read_csv(train_csv_path)
train_df.head()

# Only 14 Class

In [ ]:
train_df = train_df[train_df.class_id!=14].reset_index(drop = True)

In [ ]:
train_df['image_path'] = img_train_path+train_df.image_id+('.dicom')
train_df.head()

In [ ]:
train_df['width'] = 0
train_df['height'] = 0
train_df.head()

In [ ]:
%time
train_df['width']  = train_df.apply(lambda row: pydicom.dcmread(row.image_path).get('Columns'), axis =1)
train_df['height'] = train_df.apply(lambda row: pydicom.dcmread(row.image_path).get('Rows'), axis =1)
train_df.head()

In [ ]:
def dim_from_dicom(path):
    ds = pydicom.dcmread(path)
    return ds.get('Columns'), ds.get('Rows')

In [ ]:
%time
train_df['width'], train_df['height'] = zip(*train_df['image_path'].map(dim_from_dicom))
train_df.head()

In [ ]:
train_df['x_min'] = train_df.apply(lambda row: (row.x_min)/row.width, axis =1)
train_df['y_min'] = train_df.apply(lambda row: (row.y_min)/row.height, axis =1)

train_df['x_max'] = train_df.apply(lambda row: (row.x_max)/row.width, axis =1)
train_df['y_max'] = train_df.apply(lambda row: (row.y_max)/row.height, axis =1)

train_df['x_mid'] = train_df.apply(lambda row: (row.x_max+row.x_min)/2, axis =1)
train_df['y_mid'] = train_df.apply(lambda row: (row.y_max+row.y_min)/2, axis =1)

train_df['w'] = train_df.apply(lambda row: (row.x_max-row.x_min), axis =1)
train_df['h'] = train_df.apply(lambda row: (row.y_max-row.y_min), axis =1)

train_df['area'] = train_df['w']*train_df['h']
train_df.head()

In [ ]:
train_df.to_csv(new_csv_path)